Download Images

In [ ]:
from simple_image_download import simple_image_download as simp

# Create an instance of the Downloader class
response = simp.Downloader()

# Download images using the download method
# Note: Join the list of keywords into a single string separated by commas
keywords = ''  # If you have multiple keywords, use 'keyword1,keyword2'
response.download(keywords=keywords, limit=[])

Resize Images

In [ ]:
import os
from PIL import Image

def resize_images(directory_path, target_size=([], [])):
    for subdir, dirs, files in os.walk(directory_path):
        for filename in files:
            if any(filename.lower().endswith(ext) for ext in ['.jpeg', '.jpg', '.png', '.bmp']):
                file_path = os.path.join(subdir, filename)
                img = Image.open(file_path)
                img = img.resize(target_size, Image.LANCZOS)
                img.save(file_path)
                print(f"Resized image: {file_path}")

# Resize images in the training directory
path=''
resize_images(path)

Model training, move and rename the cloest images to selected folder

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.preprocessing import image
from sklearn.neighbors import NearestNeighbors
import shutil
from datetime import datetime

# Settings
training_directory = ''
new_images_directory = ''
save_directory = ''
valid_extensions = ['.jpeg', '.jpg', '.png', '.bmp']

# Load a pre-trained ResNet50 model trained on ImageNet without the top layer
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def extract_features(img_path, model):
    # Load and preprocess the image (assuming already resized to 224x224)
    img = image.load_img(img_path)  # No target_size here
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess the image
    # Predict the features
    features = model.predict(img_array)
    return features.flatten()

# Function to process a directory of images and extract features
def process_images(directory_path, valid_extensions):
    features_list = []
    file_paths = []
    for filename in os.listdir(directory_path):
        if any(filename.lower().endswith(ext) for ext in valid_extensions):
            file_path = os.path.join(directory_path, filename)
            file_features = extract_features(file_path, model)
            features_list.append(file_features)
            file_paths.append(file_path)
    return features_list, file_paths

# Process training and new images
training_features, training_paths = process_images(training_directory, valid_extensions)
new_features, new_paths = process_images(new_images_directory, valid_extensions)

# Train NearestNeighbors
if training_features:
    features_array = np.array(training_features)
    neighbors = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(features_array)
else:
    raise ValueError("No features were extracted from training images.")

# Find closest images
if new_features:
    new_features_array = np.array(new_features)
    distances, indices = neighbors.kneighbors(new_features_array)
else:
    raise ValueError("No features were extracted from new images.")

# Evaluate similarity and copy files
distance_threshold = np.percentile(distances, 5) # Assuming the 5th percentile corresponds to top 95% similarity

for i, distance in enumerate(distances.flatten()):
    if distance <= distance_threshold:
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
        new_basename = f"{timestamp}.jpg"
        new_name_path = os.path.join(new_images_directory, new_basename)

        # Rename and copy the image
        os.rename(new_paths[i], new_name_path)
        shutil.copy(new_name_path, os.path.join(save_directory, new_basename))
        print(f"Renamed and copied {new_name_path}")
    else:
        print(f"Image {new_paths[i]} is not similar enough to be renamed.")
